In [2]:
import pandas as pd
import numpy as np
from collections import Counter
#import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')
from umls_utils import CUIbase64,get_paths

In [77]:
!jupyter nbconvert --to script --no-prompt scHeart-Copy1.ipynb

[NbConvertApp] Converting notebook scHeart-Copy1.ipynb to script
[NbConvertApp] Writing 19705 bytes to scHeart-Copy1.py


In [2]:
!rm -r __pycache__/

In [3]:
config_path = '/Users/stearb/Dropbox/CHOP/R03/code/neo4j_build_CFDIKG/build_scripts/'


# Get paths from config file
data_dir,helper_data_dir,output_dir,LOCAL_CPU, umls_dir, umls_out_dir = get_paths(config_path)

 
if not os.path.isdir(output_dir+'scHeart'):
    os.mkdir(output_dir+'scHeart')
    print('Creating scHeart directory...')
   

In [4]:
## Flatten/melt the average gene expression data so we have 1 gene and 1 tissue per row
#df = pd.read_csv('/Users/stearb/Desktop/R03_local/data/scHeart/asp_average_gene_expression_14_celltypes.csv')
#df.rename(columns={'Unnamed: 0':'Gene'},inplace=True)
#print(df.shape)

#df2 = pd.melt(df, id_vars="Gene",value_name="Average Exp")
#df2['Average Exp'].plot(kind='hist',range=[0,10],bins=50)
#df2.to_csv('/Users/stearb/Desktop/R03_local/data/scHeart/asp_average_gene_expression_14_celltypes_LONG.csv')

In [5]:
#cui_codes_temp = pd.read_csv('/Users/stearb/Desktop/R03_local/data/scHeart/temp_files/CUI-CODEs.csv')
#cui_codes_temp

In [6]:
data_dir

'/Users/stearb/Desktop/R03_local/data/use_config/R03_DATA/'

In [7]:
z = pd.read_csv(data_dir+'asp_cell_type_markers_zeros_labelled.csv')
z.drop(['p_val','pct.1','pct.2'],axis=1,inplace=True)
print(len(z))
z.head(3)   # this data has already been filtered to remove p-vals greater than .06

2932


,avg_log2FC,p_val_adj,cluster,gene,cluster_celltype_name
0,5.513025,0.0,7,BPGM,Erythrocytes
1,5.509262,0.0,7,MT1H,Erythrocytes
2,4.710317,0.0,7,GYPB,Erythrocytes


In [8]:
#### Merge in HGNC IDs
hgnc_master = pd.read_csv(helper_data_dir+'hgnc_master.txt','\t')
gene_map = hgnc_master[['hgnc_id','symbol']].rename(columns={'symbol':'gene'})

z2 = pd.merge(z,gene_map,on='gene')

print(len(z2))

2816


In [9]:
z

,avg_log2FC,p_val_adj,cluster,gene,cluster_celltype_name
0,5.513025,0.000000,7,BPGM,Erythrocytes
1,5.509262,0.000000,7,MT1H,Erythrocytes
2,4.710317,0.000000,7,GYPB,Erythrocytes
3,3.430410,0.000000,7,EPB42,Erythrocytes
4,2.968435,0.000000,7,PHOSPHO1,Erythrocytes
...,...,...,...,...,...
2927,-1.572859,0.015851,9,POSTN,Fibroblast-like (related to larger vascular de...
2928,-1.890170,0.016514,9,MYH6,Fibroblast-like (related to larger vascular de...
2929,-1.162992,0.022877,9,IGFBP5,Fibroblast-like (related to larger vascular de...
2930,-1.384368,0.027756,9,EMCN,Fibroblast-like (related to larger vascular de...


In [11]:
# We lost rows when merging in the hgnc ids.

In [9]:
# Here are the rows we lost.
z[~z['gene'].isin(gene_map['gene'])].sample(3)

,avg_log2FC,p_val_adj,cluster,gene,cluster_celltype_name
1216,1.061135,2.844879e-22,12,ATP5C1,Myoz2-enriched cardiomyocytes
837,-1.200146,1.154896e-07,10,CTD-2545M3.8,Fibroblast-like (related to smaller vascular d...
2560,1.746059,8.345657e-42,3,FAM49B,Cardiac neural crest cells & Schwann progenito...


In [10]:
#### Merge in HGNC CUIs

# First, GET CUI - HGNC CODE MAPPINGS STRAIGHT FROM CSVs
UMLS_CUI_CODEs = pd.read_csv(umls_dir+'CUI-CODEs.csv')

umls_genes = UMLS_CUI_CODEs[UMLS_CUI_CODEs[':END_ID'].str.startswith('HGNC')].rename(
                                    columns={':START_ID':'CUI_hgnc',':END_ID':'hgnc_id'})

umls_genes['hgnc_id'] = [i.split(' ')[1] for i in umls_genes['hgnc_id']]

z3 = pd.merge(z2,umls_genes,on='hgnc_id')

print(len(z3))

2816


In [11]:
### Create the scHeart SAB
z3['SAB'] = 'scHeart PMID: 31835037'

# Create scHeart CODE

# assert that the celltype and hgncid col will uniquely identify each code node
assert len(z3) == len(z3[['cluster_celltype_name','hgnc_id']].drop_duplicates()) 

z3['CODE'] = z3['cluster_celltype_name'] + ' ' + z3['hgnc_id']

# Create scHeart CODEID
z3['CODEID'] = z3['SAB'] + ' ' + z3['CODE']

# Create scHeart CUIs
z3['CUI:ID'] = CUIbase64(z3['CODEID'])

z3.drop(['cluster','gene','hgnc_id'],axis=1,inplace=True)

z3.head(3)

,avg_log2FC,p_val_adj,cluster_celltype_name,CUI_hgnc,SAB,CODE,CODEID,CUI:ID
0,5.513025,0.000000,Erythrocytes,C1412816,scHeart PMID: 31835037,Erythrocytes HGNC:1093,scHeart PMID: 31835037 Erythrocytes HGNC:1093,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcnl0aHJvY3l0ZX...
1,-2.470632,0.010812,Epicardium-derived cells,C1412816,scHeart PMID: 31835037,Epicardium-derived cells HGNC:1093,scHeart PMID: 31835037 Epicardium-derived cell...,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcGljYXJkaXVtLW...
2,-1.961892,0.000016,Ventricular cardiomyocytes,C1412816,scHeart PMID: 31835037,Ventricular cardiomyocytes HGNC:1093,scHeart PMID: 31835037 Ventricular cardiomyocy...,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBWZW50cmljdWxhci...


### Create cell type nodes

In [12]:
# Now create the celltype nodes
z3.rename(columns={'cluster_celltype_name':'celltype_CODE'},inplace=True)

In [13]:
z3['celltype_SAB'] = 'author_defined_cluster'
z3['celltype_CODEID'] = z3['celltype_SAB'] + ':' + z3['celltype_CODE']
z3['celltype_CUI'] = CUIbase64(z3['celltype_CODEID'])

## Load in main dataset
The data_w_bins.csv file is the most up-to-date

In [14]:
#fulldata = pd.read_csv('/Users/stearb/Desktop/R03_local/data/scHeart/data_w_bins.csv')

#fulldata['CODEID'] = fulldata['SAB'] + fulldata['preCODE']

#fulldata['CUI:ID'] = CUIbase64(fulldata['CODEID'])

#fulldata.head(3)

fulldata = z3

In [15]:
### Remove rows w/ p val greater than .06 

pre=len(fulldata)

fulldata = fulldata[fulldata['p_val_adj'] < .06]

post=len(fulldata)

print(pre-post)

0


### CUIs

In [17]:
celltype_CUIs = fulldata['celltype_CUI'].rename({'celltype_CUI':'CUI:ID'}).drop_duplicates()#.reset_index(drop=True)

CUIs = pd.concat([pd.DataFrame(fulldata['CUI:ID'].values,columns=['CUI:ID']),pd.DataFrame(celltype_CUIs.values,columns=['CUI:ID'])])
CUIs.columns = ['CUI:ID']

assert CUIs.duplicated().sum() == 0
CUIs.to_csv(output_dir+'scHeart/CUIs_scHeart.csv',index=False)

### CUI_CUIs

In [18]:
sc_to_hgnc = fulldata[['CUI:ID','CUI_hgnc']]
sc_to_hgnc[':TYPE'] = 'single_cell_expression_of'
sc_to_hgnc['SAB'] = 'scHeart__HGNC'

hgnc_to_sc = fulldata[['CUI_hgnc','CUI:ID']]
hgnc_to_sc[':TYPE'] = 'has_single_cell_expression'
hgnc_to_sc['SAB'] = 'scHeart__HGNC'

sc_to_ct = fulldata[['CUI:ID','celltype_CUI']]
sc_to_ct[':TYPE'] = 'single_cell_expression_in'
sc_to_ct['SAB'] = 'scHeart__cellType'

ct_to_sc = fulldata[['celltype_CUI','CUI:ID']]
ct_to_sc[':TYPE'] = 'has_single_cell_expression'
ct_to_sc['SAB'] = 'scHeart__cellType'

CUI_CUIs = pd.DataFrame(np.concatenate([sc_to_hgnc.values,hgnc_to_sc.values , sc_to_ct.values,ct_to_sc.values
                                       ]),columns=[':START_ID',':END_ID',':TYPE','SAB'])

CUI_CUIs.drop_duplicates(inplace=True)

CUI_CUIs.to_csv(output_dir+'scHeart/CUI_CUIs_scHeart.csv',index=False)

### CUI-CODE

In [19]:
CUI_CODEs = fulldata[['CUI:ID','CODEID']]
CUI_CODEs.columns = [':START_ID',':END_ID']

CUI_CODEs_celltypes = fulldata[['celltype_CUI','celltype_CODEID']]
CUI_CODEs_celltypes.columns = [':START_ID',':END_ID']

CUI_CODEs_all = pd.concat([CUI_CODEs,CUI_CODEs_celltypes])
CUI_CODEs_all.drop_duplicates(inplace=True)
CUI_CODEs_all.to_csv(output_dir+'scHeart/CUI_CODEs_scHeart.csv',index=False)

### CODES

In [20]:
CODEs = fulldata[['CODEID','SAB','CODE']]
CODEs.columns = ['CodeID:ID','SAB','CODE']

CODEs_celltypes = fulldata[['celltype_CODEID','celltype_SAB','celltype_CODE']]
CODEs_celltypes.columns = ['CodeID:ID','SAB','CODE']

CODEs_all = pd.concat([CODEs,CODEs_celltypes])
CODEs_all.drop_duplicates(inplace=True)


CODEs_all.to_csv(output_dir+'scHeart/CODEs_scHeart.csv',index=False)

## SUIs and CODE-SUIs will both be created in 2 steps.
First create the p-value and log2fc Terms and then create the threshold Terms

### Create bins for p-values and log2fc
I  took the bins list straight from gtex, so we have the exact same bins for this data.

In [21]:
# Split p-vals up into bins

# Define pval Bins:
#### EXACT SAME BINS from GTEx notebook. ######
bins = [0,1e-12,1e-11,1e-10,1e-9,1e-8,1e-7,1e-6,1e-5,1e-4,1e-3,.005,.01,.02,.03,.04,.05,.06]

# Bin pvals
fulldata['pvalue_bins'] = pd.cut(fulldata['p_val_adj'], bins)

In [22]:
#fulldata['p_val_adj'].plot(kind='hist',bins=20)
#plt.gca().set_yscale('log')
#fulldata['avg_log2FC'].plot(kind='hist',bins=20,figsize=(10,6))

# Now bin the log2FC column

In [23]:
log2fc_bins_neg = [-5,-4,-3,-2.5,-2,-1.75,-1.5,-1.25,-1,-.75,-.5,-.25,-.2,-.15,-.1,-.05]
log2fc_bins_pos = [i*-1 for i in log2fc_bins_neg][::-1] # and reverse it.
log2fc_bins =  log2fc_bins_neg + [0] + log2fc_bins_pos + [6,7]
fulldata['log2fc_bins'] = pd.cut(fulldata['avg_log2FC'], log2fc_bins)

In [24]:
# pvalues that are 0 need to be addded to the (0.0,1e-12] bin. The lowerbound for this bin is not inclusive so
# 0's are not automatically added to it.

fulldata['pval_bins'] = [i if i is not np.nan else '(0.0,1e-12]' for i in fulldata['pvalue_bins']]
fulldata.drop('pvalue_bins',axis=1,inplace=True)

In [25]:
# Remove [] and () characters from intervals before creating SUIs
fulldata['log2fc_bins'] = [str(i)[1:-1] for i in fulldata['log2fc_bins']]
fulldata['pval_bins'] = [str(i)[1:-1] for i in fulldata['pval_bins']]

fulldata['log2fc_bins'] = fulldata['log2fc_bins'].str.replace(' ','')
fulldata['pval_bins'] = fulldata['pval_bins'].str.replace(' ','')

In [26]:
assert fulldata.isna().sum().sum() == 0
assert len(fulldata[fulldata['log2fc_bins'].isna()]) == 0

In [27]:
# Create SUIs for this step.
fulldata['SUIs_pvals'] = CUIbase64(fulldata['pval_bins'])
fulldata['SUIs_log2fc'] = CUIbase64(fulldata['log2fc_bins'])


SUIs_pvals = fulldata[['SUIs_pvals','pval_bins']].rename(columns={'SUIs_pvals':'SUI:ID','pval_bins':'name'})
SUIs_log2fc = fulldata[['SUIs_log2fc','log2fc_bins']].rename(columns={'SUIs_log2fc':'SUI:ID','log2fc_bins':'name'})


SUIs_all_bins = pd.concat([SUIs_pvals,SUIs_log2fc])
SUIs_all_bins.drop_duplicates(inplace=True)
SUIs_all_bins.reset_index(drop=True,inplace=True)

In [28]:
# Create CODE-SUIs for this step

CODE_SUIs_pvals = fulldata[['CODEID','SUIs_pvals','CUI:ID']].rename(columns={'CODEID':':START_ID','SUIs_pvals':':END_ID',
                                                          'CUI:ID':'CUI'})
CODE_SUIs_pvals[':TYPE'] = 'p_value'

In [29]:

CODE_SUIs_log2fc = fulldata[['CODEID','SUIs_log2fc','CUI:ID']].rename(columns={'CODEID':':START_ID','SUIs_log2fc':':END_ID',
                                                          'CUI:ID':'CUI'})

CODE_SUIs_log2fc[':TYPE'] = 'log2fc'

CODE_SUIs_all_bins = pd.concat([CODE_SUIs_pvals,CODE_SUIs_log2fc])

CODE_SUIs_all_bins.columns = [':START_ID', ':END_ID', 'CUI',':TYPE']

CODE_SUIs_all_bins = CODE_SUIs_all_bins[[':START_ID', ':END_ID',':TYPE','CUI']] # reorder

CODE_SUIs_all_bins.drop_duplicates(inplace=True)

# Create the Threshold Terms (they will be strings)

Thresholds:
- < .05
- < .001
- < .0001
- < 1e-10

### Assign Threshold values 

In [31]:
fulldata['pval_threshold_1e-10'] = ['< 1e-10' if i < 1e-10 else np.nan for i in fulldata['p_val_adj']]

fulldata['pval_threshold_1e-4'] = ['< 0.0001' if i < 1e-4 else np.nan for i in fulldata['p_val_adj']]

fulldata['pval_threshold_1e-3'] = ['< 0.001' if i < 1e-3 else np.nan for i in fulldata['p_val_adj']]

fulldata['pval_threshold_1e-2'] = ['< 0.01' if i < 1e-3 else np.nan for i in fulldata['p_val_adj']]

fulldata['pval_threshold_.05'] = ['< 0.05' if i < .05 else np.nan for i in fulldata['p_val_adj']]

thresh_df = fulldata[['pval_threshold_1e-10','pval_threshold_1e-4',
                  'pval_threshold_1e-3','pval_threshold_1e-2','pval_threshold_.05','p_val_adj']]

thresh_df.head(3)

,pval_threshold_1e-10,pval_threshold_1e-4,pval_threshold_1e-3,pval_threshold_1e-2,pval_threshold_.05,p_val_adj
0,< 1e-10,< 0.0001,< 0.001,< 0.01,< 0.05,0.000000
1,NaN,NaN,NaN,NaN,< 0.05,0.010812
2,NaN,< 0.0001,< 0.001,< 0.01,< 0.05,0.000016


### Create CODE-SUI file for pvals, log2fc and thresholds

In [32]:
CODE_SUIs_thresholds = pd.DataFrame(np.concatenate([fulldata[['CODEID','CUI:ID','pval_threshold_1e-10']].values,
                                        fulldata[['CODEID','CUI:ID','pval_threshold_1e-4']].values,
                                        fulldata[['CODEID','CUI:ID','pval_threshold_1e-3']].values,
                                        fulldata[['CODEID','CUI:ID','pval_threshold_1e-2']].values,
                                        fulldata[['CODEID','CUI:ID','pval_threshold_.05']].values]),
                                                 columns=['CODEID','CUI:ID','name'])

CODE_SUIs_thresholds.dropna(inplace=True)
CODE_SUIs_thresholds['SUI:ID'] = CUIbase64(CODE_SUIs_thresholds['name'])
CODE_SUIs_thresholds[':TYPE'] = 'p_value_threshold'

CODE_SUIs_thresholds.head(3)

,CODEID,CUI:ID,name,SUI:ID,:TYPE
0,scHeart PMID: 31835037 Erythrocytes HGNC:1093,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcnl0aHJvY3l0ZX...,< 1e-10,PCAxZS0xMA==,p_value_threshold
3,scHeart PMID: 31835037 Erythrocytes HGNC:7400,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcnl0aHJvY3l0ZX...,< 1e-10,PCAxZS0xMA==,p_value_threshold
4,scHeart PMID: 31835037 Erythrocytes HGNC:4703,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcnl0aHJvY3l0ZX...,< 1e-10,PCAxZS0xMA==,p_value_threshold


In [33]:
SUI_thresholds = CODE_SUIs_thresholds[['SUI:ID','name']]
SUI_thresholds.drop_duplicates(inplace=True)

In [34]:
CODE_SUIs_thresholds = CODE_SUIs_thresholds.rename(columns={'CODEID':':START_ID',
                                                            'SUI:ID':':END_ID',
                                                           'CUI:ID':'CUI'}).drop('name',axis=1)

CODE_SUIs_thresholds = CODE_SUIs_thresholds[[':START_ID',':END_ID', ':TYPE','CUI' ]]
CODE_SUIs_thresholds.head(3)

,:START_ID,:END_ID,:TYPE,CUI
0,scHeart PMID: 31835037 Erythrocytes HGNC:1093,PCAxZS0xMA==,p_value_threshold,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcnl0aHJvY3l0ZX...
3,scHeart PMID: 31835037 Erythrocytes HGNC:7400,PCAxZS0xMA==,p_value_threshold,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcnl0aHJvY3l0ZX...
4,scHeart PMID: 31835037 Erythrocytes HGNC:4703,PCAxZS0xMA==,p_value_threshold,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcnl0aHJvY3l0ZX...


In [35]:
assert CODE_SUIs_thresholds.duplicated().sum() == 0

In [36]:
CODE_SUIs_all = pd.concat([CODE_SUIs_all_bins,CODE_SUIs_thresholds])

CODE_SUIs_all.to_csv(output_dir+'scHeart/CODE_SUIs_scHeart.csv',index=False)

In [37]:
SUIs_all = pd.concat([SUIs_all_bins,SUI_thresholds])

SUIs_all.to_csv(output_dir+'scHeart/SUIs_scHeart.csv',index=False)

In [ ]:
# dont need to add p val sui's to the suis file,they are already there bc of gtex?

In [42]:
#CODE_SUIs_scHeart = pd.read_csv('/Users/stearb/Desktop/R03_local/data/scHeart/sc_data_only/CODE-SUIs.csv')

#SUIs_scHeart = pd.read_csv('/Users/stearb/Desktop/R03_local/data/scHeart/sc_data_only/SUIs.csv')

# If there are more than 4554 terms then the pval thresholds data may have already been added.
#assert len(CODE_SUIs_scHeart) == 4554

# If there are more than 35 terms then the pval thresholds data may have already been added.
#assert len(SUIs_scHeart) == 35

In [43]:
#CODE_SUIs_all = pd.concat([CODE_SUIs_scHeart,CODE_SUIs_thresholds])

#CODE_SUIs_all.to_csv('/Users/stearb/Desktop/R03_local/data/scHeart/my_scHeart_files/CODE-SUIs.csv')

In [44]:
#SUIs_all = pd.concat([SUIs_scHeart,SUI_thresholds])

#SUIs_all.to_csv('/Users/stearb/Desktop/R03_local/data/scHeart/my_scHeart_files/SUIs.csv')

In [45]:
#### SUI for '0.0,1e-12' in the graph: KDAuMCwgMWUtMTJd

#### SUI for '0.0,1e-12' in GTEx: MC4wLCAxZS0xMg==

In [46]:
#SUIs_all[SUIs_all['name'] == '0.0,1e-12']

In [62]:
#sMid = pd.read_csv('/Users/stearb/Desktop/R03_local/data/use_config/OUTPUT_FILES/scHeart/SUIs_scHeart.csv')
#sMid[sMid['SUI:ID'] == 'MWUtMTEsMWUtMTA=']

In [63]:
#sMid[sMid['SUI:ID'] == 'MC4wLDFlLTEy']

In [56]:
#SUIs_all[SUIs_all['SUI:ID'] == 'MWUtMTEsMWUtMTA='] #SUIs_all[SUIs_all['SUI:ID'] == 'MC4wLDFlLTEy']


,SUI:ID,name
4,MWUtMTEsMWUtMTA=,"1e-11,1e-10"


In [ ]:
#code_suis = pd.read_csv(umls_out_dir+'CODE-SUIs.csv')

In [55]:

#assert 'MWUtMTEsMWUtMTA=' in code_suis[':END_ID'].values
#assert 'MC4wLDFlLTEy' in code_suis[':END_ID'].values


#code_suis[code_suis[':END_ID'] == 'MWUtMTEsMWUtMTA=']
#code_suis[code_suis[':END_ID'] == 'MC4wLDFlLTEy']

,:START_ID,:END_ID,:TYPE,CUI
18121030,scHeart PMID: 31835037 Erythrocytes HGNC:1093,MC4wLDFlLTEy,p_value,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcnl0aHJvY3l0ZX...
18121033,scHeart PMID: 31835037 Erythrocytes HGNC:7400,MC4wLDFlLTEy,p_value,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcnl0aHJvY3l0ZX...
18121034,scHeart PMID: 31835037 Erythrocytes HGNC:4703,MC4wLDFlLTEy,p_value,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcnl0aHJvY3l0ZX...
18121035,scHeart PMID: 31835037 Erythrocytes HGNC:3381,MC4wLDFlLTEy,p_value,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcnl0aHJvY3l0ZX...
18121036,scHeart PMID: 31835037 Erythrocytes HGNC:16815,MC4wLDFlLTEy,p_value,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBFcnl0aHJvY3l0ZX...
...,...,...,...,...
18123841,scHeart PMID: 31835037 Fibroblast-like (relate...,MC4wLDFlLTEy,p_value,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBGaWJyb2JsYXN0LW...
18123842,scHeart PMID: 31835037 Fibroblast-like (relate...,MC4wLDFlLTEy,p_value,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBGaWJyb2JsYXN0LW...
18123843,scHeart PMID: 31835037 Fibroblast-like (relate...,MC4wLDFlLTEy,p_value,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBGaWJyb2JsYXN0LW...
18123844,scHeart PMID: 31835037 Fibroblast-like (relate...,MC4wLDFlLTEy,p_value,c2NIZWFydCBQTUlEOiAzMTgzNTAzNyBGaWJyb2JsYXN0LW...


In [76]:
#suis = pd.read_csv(umls_out_dir+'SUIs.csv')

In [78]:
#a=suis[suis['name'] == '1e-11,1e-10']
#a

In [75]:
#a['name'].values[0] == a['name'].values[1]

True

In [65]:
# gtex pval suis, sui and name

#gtex_pval_suis = pd.read_csv('/Users/stearb/Desktop/R03_local/data/use_config/OUTPUT_FILES/GTEx/SUIs_gtex.csv')

In [152]:
%%capture

'''
## Check the finished files
csv_path = '/Users/stearb/Desktop/R03_local/data/scHeart/sc_build_files/'
csv_path_check = '/Users/stearb/Desktop/R03_local/data/scHeart/sc_build_files_check/'
# !ls -lh /Users/stearb/Desktop/R03_local/data/scHeart/sc_build_files/*.csv

### CUIs  
CUIs = pd.read_csv(csv_path+'CUIs.csv')
CUIs.rename(columns={'CUI.ID':'CUI:ID'},inplace=True); print(CUIs.shape)
CUIs.to_csv('/Users/stearb/Desktop/R03_local/data/scHeart/sc_build_files_check/CUIs.csv',index=False)

CUIs_2 = pd.read_csv('/Users/stearb/Desktop/R03_local/data/scHeart/sc_build_files_check/CUIs.csv'); print(CUIs_2.shape)
len(CUIs[CUIs['CUI:ID'].isin(CUIs_2['CUI:ID'])]) == len(CUIs)
CUIs_config = pd.read_csv('/Users/stearb/Library/Application Support/com.Neo4j.Relate/Data/dbmss/dbms-320f2ff0-2090-426c-a56d-7f24f136c670/import/CUIs.csv')

### CUI-CUIs
CUI_CUIs = pd.read_csv(csv_path+'CUI-CUIs.csv')
CUI_CUIs.columns = [':START_ID',':END_ID',':TYPE','SAB']
CUI_CUIs.to_csv(csv_path_check+'CUI-CUIs.csv',index=False)
CUI_CUIs = pd.read_csv(csv_path_check+'CUI-CUIs.csv')

### CODE-SUIs
CODE_SUIs = pd.read_csv(csv_path+'CODE-SUIs.csv'); print(CODE_SUIs.shape) # 982.9 Mb
CODE_SUIs.columns = [':START_ID',':END_ID',':TYPE','SAB']
CODE_SUIs.to_csv(csv_path_check+'CODE-SUIs.csv',index=False)
CODE_SUIs = pd.read_csv(csv_path_check+'CODE-SUIs.csv'); print(CODE_SUIs.shape)
np.unique(CODE_SUIs[CODE_SUIs['SUI:ID'].duplicated()]['name']); assert 0 == sum(CODE_SUIs.duplicated())

### CUI-CODE
CODE_CUIs = pd.read_csv(csv_path+'CODE-CUIs.csv'); print(CODE_CUIs.shape)
CODE_CUIs.columns = [':START_ID',':END_ID']
CODE_CUIs.to_csv(csv_path_check+'CUI-CODEs.csv',index=False)
CUI_CODEs = pd.read_csv(csv_path_check+'CUI-CODEs.csv'); print(CUI_CODEs.shape)
CUI_CODEs_config = pd.read_csv('/Users/stearb/Library/Application Support/com.Neo4j.Relate/Data/dbmss/dbms-320f2ff0-2090-426c-a56d-7f24f136c670/import/CUI-CODEs.csv')
print(CUI_CODEs_config.shape)
CUI_CODEs_config[~CUI_CODEs_config[':START_ID'].isin(CUI_CODEs[':START_ID'])]
shared = list(set(CUI_CODEs_config[':START_ID']).intersection(CUI_CODEs[':START_ID'])); len(shared)
not_shared = CUI_CODEs_config[~CUI_CODEs_config[':START_ID'].isin(shared)]
a = [i.split(' ')[0] for i in not_shared[':END_ID']]; Counter(a)

### CODEs
CODEs = pd.read_csv(csv_path+'CODEs.csv'); print(CODEs.shape)
CODEs.rename(columns={'CodeID.ID':'CodeID:ID'},inplace=True)
CODEs.to_csv(csv_path_check+'CODEs.csv',index=False)
CODEs = pd.read_csv(csv_path_check+'CODEs.csv'); print(CODEs.shape)

### SUIs
SUIs = pd.read_csv(csv_path+'SUIs.csv'); print(SUIs.shape)
SUIs.rename(columns={'SUI.ID':'SUI:ID'},inplace=True)
SUIs.to_csv(csv_path_check+'SUIs.csv',index=False)
SUIs = pd.read_csv(csv_path_check+'SUIs.csv'); print(SUIs.shape)
SUIs.drop_duplicates(inplace=True)
SUIs.to_csv(csv_path_check+'SUIs.csv',index=False)'''

# Examine sc data only (not the master UMLS files which had the entire database + sc data)

In [41]:
latest_build_path = '/Users/stearb/Library/Application Support/com.Neo4j.Relate/Data/dbmss/dbms-320f2ff0-2090-426c-a56d-7f24f136c670/import/'
sc_only_path = '/Users/stearb/Desktop/R03_local/data/scHeart/my_scHeart_files/'


In [42]:
# Read in the master CSV files (takes a min or two)

CUIs_master = pd.read_csv(latest_build_path+'CUIs.csv')
CUI_CUIs_master = pd.read_csv(latest_build_path+'CUI-CUIs.csv')
CUI_CODEs_master = pd.read_csv(latest_build_path+'CUI-CODEs.csv')
CODEs_master = pd.read_csv(latest_build_path+'CODEs.csv')
CODE_SUIs_master = pd.read_csv(latest_build_path+'CODE-SUIs.csv')
SUIs_master = pd.read_csv(latest_build_path+'SUIs.csv')

In [67]:
# Read in the scHeart CSV files

CUIs_sc = pd.read_csv(sc_only_path+'CUIs.csv')
CUI_CUIs_sc = pd.read_csv(sc_only_path+'CUI-CUIs.csv')

CUI_CODEs_sc = pd.read_csv(sc_only_path+'CUI-CODEs.csv')
#CUI_CODEs_sc.rename(columns={':START_ID':':END_ID',':END_ID':':START_ID'},inplace=True) # switch names
#CUI_CODEs_sc = CUI_CODEs_sc[[':START_ID',':END_ID']] # reorder
#CUI_CODEs_sc.to_csv(sc_only_path+'CUI-CODEs.csv',index=False)

CODEs_sc = pd.read_csv(sc_only_path+'CODEs.csv')

CODE_SUIs_sc = pd.read_csv(sc_only_path+'CODE-SUIs.csv')
#CODE_SUIs_sc_Pval = CODE_SUIs_sc[CODE_SUIs_sc[':TYPE'] == 'p_val']
#CODE_SUIs_sc_Log2FC = CODE_SUIs_sc[CODE_SUIs_sc[':TYPE'] != 'p_val']
#CODE_SUIs_sc_Pval[':TYPE'] = 'p_value'
#CODE_SUIs_sc_FIXED = pd.concat([CODE_SUIs_sc_Pval,CODE_SUIs_sc_Log2FC])
#CODE_SUIs_sc_FIXED.to_csv(sc_only_path+'CODE-SUIs.csv',index=False)

SUIs_sc = pd.read_csv(sc_only_path+'SUIs.csv')
#SUIs_sc['name'] = [i[1:-1] for i in SUIs_sc['name']] # remove '()' and '[]' characters to match the name of the other bins
# SUIs_sc['name'] = SUIs_sc['name'].str.replace(' ','')
#SUIs_sc.drop_duplicates(inplace=True) # remove duplicate bin Terms
#SUIs_sc.to_csv(sc_only_path+'SUIs.csv',index=False)

In [69]:
Counter(CODEs_sc['SAB'])

Counter({'scHeart PMID: 31835037': 2816, 'author_defined_cluster': 14})

In [68]:
assert sum(CUIs_sc.duplicated()) == 0
assert sum(CUI_CUIs_sc.duplicated()) == 0
assert sum(CUI_CODEs_sc.duplicated()) == 0
assert sum(CODEs_sc.duplicated()) == 0
assert sum(CODE_SUIs_sc.duplicated()) == 0
#SUIs_sc.drop_duplicates(inplace=True)
assert sum(SUIs_sc.duplicated()) == 0

In [70]:
Counter(CODE_SUIs_sc[':TYPE'])

Counter({'p_value': 2816, 'log2fc': 2816, 'p_value_threshold': 13245})

# Concatenate

In [71]:
concat_CUIs = pd.concat([CUIs_master,CUIs_sc])

concat_CUI_CUIs = pd.concat([CUI_CUIs_master,CUI_CUIs_sc])

concat_CODEs =  pd.concat([CODEs_master,CODEs_sc])

concat_CUI_CODEs  = pd.concat([CUI_CODEs_master,CUI_CODEs_sc])

concat_SUIs = pd.concat([SUIs_master,SUIs_sc])
concat_SUIs.drop_duplicates(inplace=True)

concat_CODE_SUIs = pd.concat([CODE_SUIs_master,CODE_SUIs_sc])


In [72]:
assert sum(concat_CUIs.duplicated()) == 0
assert sum(concat_CUI_CUIs.duplicated()) == 0
assert sum(concat_CUI_CODEs.duplicated()) == 0
assert sum(concat_CODEs.duplicated()) == 0
assert sum(concat_CODE_SUIs.duplicated()) == 0
assert sum(concat_SUIs.duplicated()) == 0

# Save the updated master files

In [73]:
sc_heart_build_path = '/Users/stearb/Desktop/R03_local/data/scHeart/sc_heart_build/'

In [74]:
concat_CUIs.to_csv(sc_heart_build_path+'CUIs.csv',index=False)
concat_CUI_CUIs.to_csv(sc_heart_build_path+'CUI-CUIs.csv',index=False)
concat_CODEs.to_csv(sc_heart_build_path+'CODEs.csv',index=False)
concat_CUI_CODEs.to_csv(sc_heart_build_path+'CUI-CODEs.csv',index=False)
concat_SUIs.to_csv(sc_heart_build_path+'SUIs.csv',index=False)
concat_CODE_SUIs.to_csv(sc_heart_build_path+'CODE-SUIs.csv',index=False)

In [64]:
concat_SUIs.head()

,SUI:ID,name
0,S17175117,"1,2-dipalmitoylphosphatidylcholine"
1,S0007620,1-Aminonaphthalene
2,S11842225,17-ketogenic steroid
3,S0007902,"2',3' Cyclic Nucleotide Phosphodiesterases"
4,S2029232,"Phosphodiesterases, 2',3'-Cyclic Nucleotide"


In [76]:
CODEs_sc

,CodeID:ID,SAB,CODE
0,scHeart PMID: 31835037 Erythrocytes HGNC:1093,scHeart PMID: 31835037,Erythrocytes HGNC:1093
1,scHeart PMID: 31835037 Epicardium-derived cell...,scHeart PMID: 31835037,Epicardium-derived cells HGNC:1093
2,scHeart PMID: 31835037 Ventricular cardiomyocy...,scHeart PMID: 31835037,Ventricular cardiomyocytes HGNC:1093
3,scHeart PMID: 31835037 Erythrocytes HGNC:7400,scHeart PMID: 31835037,Erythrocytes HGNC:7400
4,scHeart PMID: 31835037 Erythrocytes HGNC:4703,scHeart PMID: 31835037,Erythrocytes HGNC:4703
...,...,...,...
2825,author_defined_cluster:Cardiac neural crest ce...,author_defined_cluster,Cardiac neural crest cells & Schwann progenito...
2826,author_defined_cluster:Fibroblast-like (relate...,author_defined_cluster,Fibroblast-like (related to larger vascular de...
2827,author_defined_cluster:Epicardial cells,author_defined_cluster,Epicardial cells
2828,author_defined_cluster:Myoz2-enriched cardiomy...,author_defined_cluster,Myoz2-enriched cardiomyocytes


In [162]:
concat_SUIs.nunique()

SUI:ID    11526257
name      11517015
dtype: int64

In [164]:
SUIs_master.nunique()

SUI:ID    11526243
name      11516981
dtype: int64

In [165]:
len(SUIs_master)

11526243

In [169]:
nameDupsuis = SUIs_master[SUIs_master['name'].duplicated()]

In [174]:
nameDupsuis.reset_index(drop=True,inplace=True)

In [188]:
nameDupsuis[~nameDupsuis['SUI:ID'].str.startswith('S')]       #.iloc[:-12,:]

,SUI:ID,name
101,ZXhvc3Rvc2luIGdseWNvc3lsdHJhbnNmZXJhc2UgMg==,exostosin glycosyltransferase 2
102,YW5uZXhpbiBBOA==,annexin A8
103,YXJnaW5pbmUgdmFzb3ByZXNzaW4gcmVjZXB0b3IgMUI=,arginine vasopressin receptor 1B
104,Zm9ya2hlYWQgYm94IE8x,forkhead box O1
105,Z3Jvd3RoIGRpZmZlcmVudGlhdGlvbiBmYWN0b3IgMTU=,growth differentiation factor 15
...,...,...
9257,MTAxMQ==,1011
9258,NDc1,475
9259,NzU1,755
9260,Mzgx,381


In [184]:
SUIs_master[SUIs_master['SUI:ID'] == 'NDc1']

,SUI:ID,name
11497049,NDc1,475


In [185]:
SUIs_master[SUIs_master['name'] == '475']

,SUI:ID,name
4881528,S17240993,475
11497049,NDc1,475


In [190]:
CODE_SUIs_master[CODE_SUIs_master[':END_ID']]

,:START_ID,:END_ID,:TYPE,CUI
5127010,SNOMEDCT_US 732760005,S17240993,PT,C4517786


In [191]:
CODE_SUIs_master[CODE_SUIs_master[':END_ID'] == 'S17240993']

,:START_ID,:END_ID,:TYPE,CUI
5127010,SNOMEDCT_US 732760005,S17240993,PT,C4517786
